In [1]:
# Dependencies and Setup
from pathlib import Path
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete


,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


## Local Government Area Summary

In [2]:
# Calculate the Totals (Schools and Students)
school_count = school_data_complete['school_name'].nunique()
student_count = school_data_complete["student_name"].count()

# Calculate the Total Budget
# Did for each unique school
# add .reset_index to add index on left.
#total_budget = school_data_complete.groupby('school_name')['budget'].sum().reset_index()

#for all schools THIS GIVES WRONG AMOUNT
#overall_budget = school_data_complete['budget'].sum()

#unique_budget_count = school_data_complete['school_name'].value_counts()
#budget_groups = school_data_complete.groupby('budget')['school_name'].apply(list).reset_index()

#first we calculate all unique values for budget, them we total all those budgets
unique_budgets = school_data_complete['budget'].unique()
overall_budget = unique_budgets.sum()


#student_count
#school_count
#total_budget
overall_budget
#unique_budgets



24649428

In [3]:
# Calculate the Average Scores
average_maths_score = school_data_complete['maths_score'].mean()
average_reading_score = school_data_complete['reading_score'].mean()

average_maths_score
#average_reading_score


70.33819249425581

In [4]:
# Calculate the Percentage Pass Rates
# check for students who passes over 50
passing_maths_count = school_data_complete[(school_data_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

#check students who passed for both reading and maths
passing_maths_reading_count = school_data_complete[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)].count()["student_name"]

#Adding the counts of students who passed math and reading and subtracting those who were counted twice (those who passed both).
# if we remove the subtract part, count will double up, counting total students that passed
overall_passing_count = passing_maths_count + passing_reading_count - passing_maths_reading_count

#percentage of students who passed/total students
overall_passing_rate = passing_maths_reading_count / float(student_count) * 100

overall_passing_rate
#overall_passing_count
#passing_maths_percentage
#passing_reading_percentage

72.80827163645647

In [5]:
#create dictionary
data = {
    "Total Students": [student_count],
    "Total Budget": [overall_budget],
    "Average Math Score": [average_maths_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Maths": [passing_maths_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_rate]
}

# Convert to DataFrame
area_summary = pd.DataFrame(data)
#print(area_summary.dtypes)
#print(area_summary["Total Budget"])

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary


,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

In [6]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]

#check values inside this, tried to count students per school without looking what was inside
print(school_data.columns)


# Calculate the total student count per school from school_data
per_school_counts = school_data.groupby('school_name')['size'].sum().reset_index()
#per_school_counts   .value_counts()

#per_school_counts = school_data.groupby('school_name')['size'].sum().reset_index()


#print(per_school_counts)


# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data.groupby('school_name')['budget'].sum().reset_index()

#per_school_budget = school_data.groupby('school_name')['budget'].sum().reset_index()

#print(per_school_budget)    

#divide the total school budget by the total number of students for each school
per_school_capita = per_school_budget['budget'] / per_school_counts['size']
print("per capita budget")
#print(per_school_capita) #if we leave this uncommented will give error when making next dataframe

#####################################
#merge columns
#per_school_data = pd.merge(per_school_budget, per_school_counts, on='school_name')
# Calculate per capita spending and create a new column
#per_school_data['school capita'] = per_school_data['budget'] / per_school_data['size']
#per_school_data
#####################################

# Calculate the average test scores per school from school_data_complete
per_school_maths = school_data_complete.groupby('school_name')['maths_score'].mean().reset_index()
per_school_reading = school_data_complete.groupby('school_name')['reading_score'].mean().reset_index() 

#print(per_school_maths)
#per_school_reading
#school_types

Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')
per capita budget


In [7]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.
school_passing_maths = school_data_complete[school_data_complete['maths_score'] >= 50]
school_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 50]

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.

# Count the number of students passing maths and reading by school
passing_maths_count = school_passing_maths.groupby('school_name').size()
passing_reading_count = school_passing_reading.groupby('school_name').size()

#passing_maths_and_reading = school_data_complete[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)].count()["student_name"]
#passing_maths_and_reading

# Get the count of students who passed both subjects
passing_both_count = school_data_complete[(school_data_complete["maths_score"] >= 50) & 
                                          (school_data_complete["reading_score"] >= 50)].groupby('school_name').size()

passing_both_count
#passing_maths_count
#passing_reading_count

#school_passing_maths
#school_passing_reading


school_name
Bailey High School       3985
Cabrera High School      1501
Figueroa High School     1995
Ford High School         1848
Griffin High School      1194
Hernandez High School    3076
Holden High School        337
Huang High School        1946
Johnson High School      3199
Pena High School          762
Rodriguez High School    3176
Shelton High School      1389
Thomas High School       1136
Wilson High School       1540
Wright High School       1435
dtype: int64

In [8]:
# Total number of students per school
per_school_counts = school_data_complete['school_name'].value_counts()

#getting names of schools only
per_school_namesOnly = school_data_complete['school_name'].unique()

#print(per_school_namesOnly)

#was having errors, with per school count index, it was not getting the unique values
#index mismatched - it was not retrieving the actual school names
#which lead to my dataframe only printing NaNs
#fixed by adding per_school_counts = school_data_complete['school_name'].value_counts()

#print("Passing Maths Count Index:\n", passing_maths_count.index)
#print("Passing Reading Count Index:\n", passing_reading_count.index)
#print("Per School Counts Index:\n", per_school_counts.index)

# Check the output
#print("Per School Counts:\n", per_school_counts)

#  Calculate the Percentage Pass Rates
per_school_passing_maths = (passing_maths_count / per_school_counts) * 100
per_school_passing_reading = (passing_reading_count / per_school_counts) * 100


overall_passing_rate = (passing_both_count / per_school_counts) * 100



print(per_school_passing_maths)
#print("Maths Passing rate:\n", per_school_passing_maths)
#print("Reading Passing rate:\n", per_school_passing_reading)
#per_school_passing_maths
#per_school_counts

school_name
Bailey High School       91.639871
Cabrera High School      90.850377
Figueroa High School     81.654798
Ford High School         82.438846
Griffin High School      91.212534
Hernandez High School    80.949299
Holden High School       89.929742
Huang High School        81.693521
Johnson High School      82.062592
Pena High School         91.683992
Rodriguez High School    90.797699
Shelton High School      91.538898
Thomas High School       83.853211
Wilson High School       82.785808
Wright High School       91.777778
dtype: float64


In [10]:
#print(per_school_capita)
#per_school_capita not in correct order

#BE CAREFUL, THIS PART GET ERRORS ALOT, If running this section, MAKE SURE TO RESTART KERNEL.

# Create lists for the summary DataFrame
per_school_counts = school_data_complete['school_name'].value_counts().reindex(per_school_namesOnly).fillna(0).astype(int).tolist()
per_school_budget = school_data.groupby('school_name')['budget'].sum().reindex(per_school_namesOnly).fillna(0).tolist()
#per_school_capita = per_school_budget / per_school_counts  # This will need to be adjusted to avoid division by zero
per_school_maths = school_data_complete.groupby('school_name')['maths_score'].mean().reindex(per_school_namesOnly).fillna(0).tolist()
per_school_reading = school_data_complete.groupby('school_name')['reading_score'].mean().reindex(per_school_namesOnly).fillna(0).tolist()

#per_school_capita = per_school_passing_maths.reindex(per_school_namesOnly).tolist()
per_school_passing_maths = per_school_passing_maths.reindex(per_school_namesOnly).fillna(0).tolist()
per_school_passing_reading = per_school_passing_reading.reindex(per_school_namesOnly).fillna(0).tolist()
overall_passing_rate = overall_passing_rate.reindex(per_school_namesOnly).fillna(0).tolist()

#per_school_passing_maths = (passing_maths_count / per_school_counts) * 100
#per_school_passing_reading = (passing_reading_count / per_school_counts) * 100



data = {
    "School Name": per_school_namesOnly,  
    "School Type": school_types,  # Ensure this is a list of the same length
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita.tolist(),  # Convert to list
    "Average Maths Score": per_school_maths,
    "Average Reading Score": per_school_reading,
    "% Passing Maths": per_school_passing_maths,  # Convert to list
    "% Passing Reading": per_school_passing_reading,
    "% Overall passing": overall_passing_rate
}

# Convert to DataFrame
per_school_summary = pd.DataFrame(data)

per_school_summary.index = per_school_summary.index.str.strip().str.lower()  # Standardize school names


# Set the index to 'School Name' and sort
per_school_summary.set_index('School Name', inplace=True)
per_school_summary.sort_index(ascending=True, inplace=True)


# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)


per_school_summary


# Display the DataFrame
#print(per_school_summary)

AttributeError: 'list' object has no attribute 'reindex'

## Top Performing Schools (By % Overall Passing)

In [ ]:
# Sort and show top five schools
top_schools =
top_schools.head(5)


## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort and show bottom five schools
bottom_schools =
bottom_schools.head(5)


## Maths Scores by Year

In [ ]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores =
year_ten_scores =
year_eleven_scores =
year_twelve_scores =

# Combine series into single DataFrame
maths_scores_by_year =

# Minor data wrangling
maths_scores_by_year.index.name = None

# Display the DataFrame
maths_scores_by_year


## Reading Score by Year

In [ ]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores =
year_ten_scores =
year_eleven_scores =
year_twelve_scores =

# Combine series into single DataFrame
reading_scores_by_year =

# Minor data wrangling
reading_scores_by_year.index.name = None

# Display the DataFrame
reading_scores_by_year


## Scores by School Spending

In [ ]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]


In [ ]:
# Create a copy of the school summary for later aggregations
# This step can be skipped but it's best to make a copy.
school_spending_df = per_school_summary.copy()


In [ ]:
# Use `pd.cut` on the per_school_capita Series from earlier to categorise per student spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] =

# Convert Spending Ranges (Per Student) to a string
school_spending_df["Spending Ranges (Per Student)"] = school_spending_df["Spending Ranges (Per Student)"].astype(str)
school_spending_df


In [ ]:
#  Calculate averages for the desired columns.
spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_maths = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


In [ ]:
# Assemble into DataFrame
spending_summary =

# Display results
spending_summary


## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [ ]:
# Create a copy of the school summary for later aggregations
# This step can be skipped but it's best to make a copy.
school_size_df = per_school_summary.copy()


In [ ]:
# Use `pd.cut` on the per_school_counts Series from earlier to categorise school size based on the bins.
school_size_df["School Size"] =

# Convert School Size to a string
school_size_df["School Size"] = school_size_df["School Size"].astype(str)
school_size_df

In [ ]:
# Calculate averages for the desired columns.
size_maths_scores = school_size_df.groupby(["School Size"])["Average Maths Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_maths = school_size_df.groupby(["School Size"])["% Passing Maths"].mean()
size_passing_reading = school_size_df.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["School Size"])["% Overall Passing"].mean()


In [ ]:
# Assemble into DataFrame
size_summary =

# Display results
size_summary


## Scores by School Type

In [ ]:
# Create new series using groupby for:
# Type | Average Maths Score | Average Reading Score | % Passing Maths | % Passing Reading | % Overall Passing

type_maths_scores = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_maths = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
type_overall_passing = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()


In [ ]:
# Assemble into DataFrame
type_summary =

# Display results
type_summary
